In [1]:
# inport BeautifulSoup
import re
from bs4 import BeautifulSoup, Comment

In [2]:
folder = "20170705"
folder

'20170705'

In [3]:
# chrome_match.html
# firefox_player_skill_table.txt
file = open(folder + "/chrome_match.html", "r")
soup = BeautifulSoup(file.read(), "html.parser")
file.close()

In [5]:
for div in soup.find_all("div", id="ctl00_ctl00_CPContent_CPMain_ucPostMatch_rptTimeline_ctl14_timelineEventPanel"):
    #for inputTag in div.find_all("input", id="ctl00_ctl00_CPContent_CPMain_ucPostMatch_rptTimeline_ctl14_playerRatingsAway"):
    for inputTag in div.find_all("input", id="ctl00_ctl00_CPContent_CPMain_ucPostMatch_rptTimeline_ctl14_playerRatingsHome"):
        print(inputTag.prettify())

<input id="ctl00_ctl00_CPContent_CPMain_ucPostMatch_rptTimeline_ctl14_playerRatingsHome" name="ctl00$ctl00$CPContent$CPMain$ucPostMatch$rptTimeline$ctl14$playerRatingsHome" type="hidden" value='[{"PlayerId":427619232,"PositionID":0,"PositionBehaviour":0,"Stars":0.0,"InjuryLevel":0,"Stamina":0.0,"Cards":0,"IsCaptain":false,"IsKicker":false,"FromMin":-1,"ToMin":-1},{"PlayerId":427619233,"PositionID":100,"PositionBehaviour":0,"Stars":4.0,"InjuryLevel":0,"Stamina":0.939741969,"Cards":0,"IsCaptain":false,"IsKicker":false,"FromMin":-1,"ToMin":91},{"PlayerId":427619238,"PositionID":101,"PositionBehaviour":0,"Stars":4.0,"InjuryLevel":0,"Stamina":0.953209043,"Cards":0,"IsCaptain":false,"IsKicker":false,"FromMin":-1,"ToMin":91},{"PlayerId":427619235,"PositionID":102,"PositionBehaviour":0,"Stars":3.5,"InjuryLevel":0,"Stamina":0.9475181,"Cards":0,"IsCaptain":false,"IsKicker":false,"FromMin":-1,"ToMin":91},{"PlayerId":427619239,"PositionID":104,"PositionBehaviour":0,"Stars":3.0,"InjuryLevel":0,"Sta

In [ ]:
import psycopg2

In [ ]:
def dropPlayer():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """DROP TABLE IF EXISTS player ; """
        cur.execute(sql)
        conn.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
# dropPlayer()

In [ ]:
def createPlayer():
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """
CREATE TABLE player (
    Date DATE, 
    Name VARCHAR(30), 
    Age VARCHAR(10), 
    TSI BIGINT, 
    PayPerWeek BIGINT, 
    Form INT, 
    Stamina INT, 
    Special VARCHAR(8),
    Experience INT,
    Leadership INT,
    Loyalty INT,
    Keeper INT,
    Defending INT,
    Playmaking INT,
    Winger INT,
    Passing INT,
    Scoring INT,
    SetPieces INT
) ; """
        cur.execute(sql)
        conn.commit()
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
#createPlayer()

In [ ]:
def insertPlayer(folder, row):
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        sql = """INSERT INTO player (SELECT to_date(%s, 'YYYYMMDD'), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ;"""
        cur.execute(sql, (folder,row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13],row[14],row[15],row[16]))
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
def selectPlayer(tableName):
    conn = None
    try:
        conn = psycopg2.connect("dbname='mydatabase' user='myuser' host='localhost' port='65432' password='123qwe'")
        cur = conn.cursor()
        cur.execute("SELECT * FROM " + tableName)
        row = cur.fetchone()
        while row is not None:
            print(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
            print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
#selectPlayer("player")

In [ ]:
import csv

In [ ]:
with open(folder + '/player.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if(re.match('^[0-9]{1,2}\. ', row[0])) :
            print(folder, row)
            insertPlayer(folder, row)